**<center><h1>EXTRACCIÓN DE DATOS</h1></center>**

La extracción de datos es una de las tareas más importantes en el análisis de datos. La extracción de datos se refiere a la recuperación de información de una o varias fuentes de datos y su almacenamiento en un formato que pueda ser utilizado para análisis. <br>Puede ser el primer paso en la creación de un conjunto de datos que se utilizará para análisis posteriores.  <br><br>
Existen varias herramientas y técnicas que pueden utilizarse para la extracción de datos en análisis de datos, incluyendo: 
*   Web Scraping
*   Bases de Datos
*   Archivos de Texto
*   APIs
*   Otros
<br><br>

En este notebook, aprenderás cómo extraer datos de páginas web y documentos utilizando técnicas de webscraping en Python. 

**Webscraping** es una técnica que nos permite recuperar información estructurada de una página web y almacenarla en un formato que podamos utilizar para análisis posteriores.

Inicialmente vamos a obtener los datos de un pdf el cual debes subir en el espacio correspondiente.

# **Extracción de datos desde un documento**

In [4]:
#Instalar la librería PyPDF2
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#Importamos las librerías
import PyPDF2
import openpyxl

In [6]:
# Crear un archivo de Excel
wb = openpyxl.Workbook()

In [7]:
# Crear una hoja de trabajo
ws = wb.active

In [8]:
# Escribir encabezados de columnas
ws['A1'] = 'Documento'
ws['B1'] = 'Nombre'
ws['C1'] = 'Apellidos'
ws['D1'] = 'Área de residencia'
ws['E1'] = 'Tipo colegio'
ws['F1'] = 'Género'
ws['G1'] = 'Estrato'
ws['H1'] = 'Nota Matematicas'
ws['I1'] = 'Nota Lecto Escritura'
ws['J1'] = 'Apoyo económico de padres'
ws['K1'] = 'Último grado cursado'

In [10]:
#Abrir y recorrer el archivo pdf
with open('Datos Aprendices.pdf', 'rb') as pdf_file:
    # Crear un objeto de lector de PDF
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Obtener el número de páginas del PDF
    num_paginas = len(pdf_reader.pages)

    # Iterar sobre todas las páginas y extraer el texto
    for pagina in range(num_paginas):
        page = pdf_reader.pages[pagina]
        text = page.extract_text()   
        matriz = text.split()
        
        row = ws.max_row + 1

        for i in range(len(matriz)):
            if "Documento" in matriz[i]:
                documento = matriz[i+1]
                ws.cell(row=row, column=1, value=documento)
            
            if "Nombre" in matriz[i]:
                nombre = matriz[i+1]
                ws.cell(row=row, column=2, value=nombre)
            
            if "Apellidos" in matriz[i]:
                apellidos = matriz[i+1]
                ws.cell(row=row, column=3, value=apellidos)
            
            if "residencia" in matriz[i]:
                area = matriz[i+1]
                ws.cell(row=row, column=4, value=area)
            
            if "colegio" in matriz[i]:
                colegio = matriz[i+1]
                ws.cell(row=row, column=5, value=colegio)
            
            if "Género" in matriz[i]:
                genero = matriz[i+1]
                ws.cell(row=row, column=6, value=genero)
            
            if "Estrato" in matriz[i]:
                estrato = matriz[i+1]
                ws.cell(row=row, column=7, value=estrato)
            
            if "Matemáticas" in matriz[i]:
                matematicas = matriz[i+1]
                ws.cell(row=row, column=8, value=matematicas)
            
            if "Escritura" in matriz[i]:
                lecto = matriz[i+1]
                ws.cell(row=row, column=9, value=lecto)
            
            if "padres" in matriz[i]:
                padres = matriz[i+1]
                ws.cell(row=row, column=10, value=padres)
            
            if "cursad" in matriz[i]:
                grado = matriz[i+2]
                ws.cell(row=row, column=11, value=grado)   

In [11]:
# Escribir el archivo de Excel        
wb.save('datos_pdf.xlsx') 

# **Extracción de datos de una web**

In [12]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
# Descargar la página web
url = "https://ingdanielbs.github.io/Datos/"
response = requests.get(url)

In [14]:
# Analizar el HTML con BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [15]:
# Encontrar la tabla y obtener los datos
table = soup.find("table")

In [17]:
if table is not None:
    rows = table.find_all("tr")
    # Resto del código aquí
    data = []
    for row in rows:
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    # Convertir los datos en un DataFrame de Pandas
    df = pd.DataFrame(data, columns=["Documento", "Nombre", "Apellidos", "Direccción", "Teléfono", "Edad", "Estatura"])

    # Escribir el DataFrame en un archivo de Excel
    df.to_excel("datos.xlsx", index=False)
else:
    print("La tabla no fue encontrada")

A continuación vamos a generar un nuevo dataset con la unión de los archivos creados anteriormente.

In [18]:
# Cargar los datos de ambos archivos
datos = pd.read_excel('datos.xlsx')
datos_pdf = pd.read_excel('datos_pdf.xlsx')

# Unir los datos basados en la columna "Documento", especificando qué columnas incluir
datos_unidos = pd.merge(datos, datos_pdf[['Documento', 'Área de residencia', 'Tipo colegio', 'Género', 'Estrato', 'Nota Matematicas', 'Nota Lecto Escritura', 'Apoyo económico de padres', 'Último grado cursado']], on='Documento')

# Guardar los datos unidos en un nuevo archivo de Excel
datos_unidos.to_excel('Dataset_final.xlsx', index=False)